In [7]:
df

,Day_Index,Is_Weekend,Total_Admissions_Today,Total_Discharges_Today,ICU_Patients_Today,Surgeries_Today,Total_Beds_Occupied_Today,Total_Beds_Required_Tomorrow,Doctors_Required_Tomorrow,Nurses_Required_Tomorrow,Technicians_Required_Tomorrow
0,0,False,43,24,8,1,139,136,13,31,3
1,1,False,37,20,9,2,156,156,13,31,5
2,2,False,16,33,3,3,139,138,7,17,4
3,3,False,47,21,10,0,165,170,14,33,3
4,4,False,42,25,8,3,182,185,15,35,5
...,...,...,...,...,...,...,...,...,...,...,...
725,725,False,33,14,4,0,4362,4359,8,20,3
726,726,True,31,20,3,0,4373,4378,7,18,3
727,727,True,21,18,9,2,4376,4379,10,23,5
728,728,False,33,14,10,3,4395,4390,14,32,6


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os
import pickle
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("staff_data.csv")

# Convert Date column to datetime and create Is_Weekend feature
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"])
    df["Is_Weekend"] = df["Date"].dt.dayofweek.isin([5, 6]).astype(int)  # 1 for Sat/Sun, 0 otherwise
    df.set_index("Date", inplace=True)

# Define features for prediction
features = [
    "Total_Admissions_Today",
    "Total_Discharges_Today",
    "ICU_Patients_Today",
    "Surgeries_Today",
    "Total_Beds_Occupied_Today",
    "Total_Beds_Required_Tomorrow",  # Using predicted bed requirement
    "Is_Weekend"
]

# Targets (Staff predictions)
targets = ["Doctors_Required_Tomorrow", "Nurses_Required_Tomorrow", "Technicians_Required_Tomorrow"]

# Normalize numerical features (excluding Is_Weekend)
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[features + targets] = scaler.fit_transform(df[features + targets])  # Scale all numerical data
df_scaled = df_scaled.astype(np.float32)

# Train-Test Split (80% Train, 20% Test)
train_size = int(len(df_scaled) * 0.8)
train, test = df_scaled.iloc[:train_size], df_scaled.iloc[train_size:]

# Function to create sequences for LSTM
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length][features].values)  # Features
        y.append(data.iloc[i+seq_length][targets].values)  # Targets (staff predictions)
    return np.array(X), np.array(y)

# Prepare sequences
seq_length = 10  # Model looks at last 10 days
X_train, y_train = create_sequences(train, seq_length)
X_test, y_test = create_sequences(test, seq_length)

# Define LSTM Model
def build_lstm_model():
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(seq_length, len(features))),
        Dropout(0.2),
        LSTM(128),
        Dropout(0.1),
        Dense(len(targets))  # Output layer predicts all 3 staff categories
    ])
    model.compile(loss='mse', optimizer='adam')
    return model

# Path for saving model
model_path = "lstm_staff_prediction.pkl"

# Load or Train Model
if os.path.exists(model_path):
#     pass
#     # with open(model_path, "rb") as f:
#     #     model_json, weights = pickle.load(f)
#     # model = model_from_json(model_json)
#     # model.set_weights(weights)
#     # model.compile(loss='mse', optimizer='adam')
#     # print("Loaded existing model.")
# else:
    model = build_lstm_model()
    model.fit(X_train, y_train, epochs=200, batch_size=16, validation_data=(X_test, y_test), verbose=1)

    # Save model
    model_json = model.to_json()
    weights = model.get_weights()
    with open(model_path, "wb") as f:
        pickle.dump((model_json, weights), f)

    print("Trained and saved new model.")

# Predict next day's staff requirement
latest_data = df_scaled.iloc[-seq_length:][features].values.reshape(1, seq_length, len(features))
predicted_scaled = model.predict(latest_data)

# Reverse scale the prediction
predicted_full = np.zeros((1, len(features) + len(targets)))
predicted_full[:, -len(targets):] = predicted_scaled.flatten()  # Insert predictions in target positions
predicted_staff = scaler.inverse_transform(predicted_full)[:, -len(targets):][0]  # Extract only target values

# Display predicted staff numbers
print(f"Predicted Doctors Required for Tomorrow: {predicted_staff[0]:.2f}")
print(f"Predicted Nurses Required for Tomorrow: {predicted_staff[1]:.2f}")
print(f"Predicted Technicians Required for Tomorrow: {predicted_staff[2]:.2f}")

# Function to update model with actual values
# def update_model_with_actual(actual_staff):
#     """
#     Updates the model with new actual data at the end of the day.
#     """
#     global df_scaled, X_train, y_train

#     # Add new actual data
#     new_data = df.iloc[-1:].copy()
#     new_data[targets] = actual_staff  # Use actual entered values

#     # Normalize new data
#     new_data_scaled = new_data.copy()
#     new_data_scaled[features + targets] = scaler.transform(new_data[features + targets])

#     # Append new data
#     df_scaled = pd.concat([df_scaled, new_data_scaled])

#     # Re-create sequences
#     X_train, y_train = create_sequences(df_scaled, seq_length)

#     # Fine-tune model with the new data
#     model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=1)

#     # Save updated model
#     model_json = model.to_json()
#     weights = model.get_weights()
#     with open(model_path, "wb") as f:
#         pickle.dump((model_json, weights), f)

#     print("Model updated with new data.")

# # Get actual values entered by hospital staff
# actual_doctors = float(input("Enter actual doctors required for today: "))
# actual_nurses = float(input("Enter actual nurses required for today: "))
# actual_technicians = float(input("Enter actual technicians required for today: "))

# # Compare and update model
# print(f"Predicted Doctors: {predicted_staff[0]:.2f}, Actual: {actual_doctors:.2f}")
# print(f"Predicted Nurses: {predicted_staff[1]:.2f}, Actual: {actual_nurses:.2f}")
# print(f"Predicted Technicians: {predicted_staff[2]:.2f}, Actual: {actual_technicians:.2f}")

# update_model_with_actual([actual_doctors, actual_nurses, actual_technicians])


Epoch 1/200


c:\Users\HARSHDIP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
from sklearn.metrics import r2_score

# Make predictions on the test set
y_pred_scaled = model.predict(X_test)

# Reverse scale the predictions
y_pred_full = np.zeros((len(y_pred_scaled), len(features) + len(targets)))
y_pred_full[:, -len(targets):] = y_pred_scaled  # Place predictions in target positions
y_pred = scaler.inverse_transform(y_pred_full)[:, -len(targets):]  # Extract only target values

# Reverse scale actual test values
y_test_full = np.zeros((len(y_test), len(features) + len(targets)))
y_test_full[:, -len(targets):] = y_test  # Place actual values in target positions
y_actual = scaler.inverse_transform(y_test_full)[:, -len(targets):]  # Extract only target values

# Compute R² score for each staff type
r2_doctors = r2_score(y_actual[:, 0], y_pred[:, 0])
r2_nurses = r2_score(y_actual[:, 1], y_pred[:, 1])
r2_technicians = r2_score(y_actual[:, 2], y_pred[:, 2])

# Display R² scores
print(f"R² Score for Doctors Prediction: {r2_doctors:.4f}")
print(f"R² Score for Nurses Prediction: {r2_nurses:.4f}")
print(f"R² Score for Technicians Prediction: {r2_technicians:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step  
R² Score for Doctors Prediction: -0.5748
R² Score for Nurses Prediction: -0.6175
R² Score for Technicians Prediction: -0.4923
